In [6]:
# coding: utf-8
import os
# import cv2 as cv
import numpy as np
import colorsys
import os
import random
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
# from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners
from PIL import Image, ImageDraw, ImageFont

class YOLO():
    def __init__(self,model_path = 'F://yolo.h5',image_shape = (480., 640.),max_boxes=10,score_threshold = 0.3,iou_threshold = 0.5):
        self.model_path = model_path
        self.image_shape = image_shape
        self.max_boxes = max_boxes
        self.score_threshold = score_threshold
        self.iou_threshold = iou_threshold

        self.class_names = self._read_classes()
        self.anchors = self._read_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _read_classes(self):
        class_names = ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier','toothbrush']
        return class_names

    def _read_anchors(self):
        anchors = np.array([0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]).reshape(-1, 2)
        return anchors

    def generate(self):
#         model_path = os.path.expanduser(self.model_path)
#         assert model_path.endswith('.h5'), 'Keras model must be a .h5 file.'
        self.yolo_model = load_model(self.model_path)

        yolo_outputs = yolo_head(self.yolo_model.output, self.anchors, len(self.class_names))
        scores, boxes, classes = self.yolo_eval(yolo_outputs, self.image_shape,max_boxes=self.max_boxes,score_threshold=self.score_threshold, iou_threshold=self.iou_threshold)
        return boxes, scores, classes
    
def yolo_boxes_to_corners(box_xy,box_wh):
    box_mins = box_xy - (box_wh/2.)
    box_maxes = box_xy + (box_wh/2.)
    return K.concatenate([box_mins[...,1:2],
                         box_mins[...,0:1],
                         box_maxes[...,1:2],
                         box_maxes[...,0:1]])
    
    def yolo_head(feats, anchors, num_classes):
        num_anchors = len(anchors)
        anchors_tensor = K.reshape(K.variable(anchors), [1,1,1,num_anchors,2])

        conv_dims = K.shape(feats)[1:3]
        conv_height_index = K.arange(0, stop=conv_dims[0])
        conv_width_index = K.arange(0, stop=conv_dims[1])
        conv_height_index = K.tile(conv_height_index, [conv_dims[1]])

        conv_width_index = K.tile(K.expand_dims(conv_width_index,0), [conv_dims[0], 1])
        conv_width_index = K.flatten(K.transpose(conv_width_index))
        conv_index = K.transpose(K.stack([conv_height_index, conv_width_index]))
        conv_index = K.reshape(conv_indexx, [1, conv_dims[0], conv_dims[1], 1, 2])
        conv_index = K.cast(conv_index, K.dtype(feats))

        feats = K.reshape(feats, [-1, conv_dims[0], conv_dims[1], num_anchors, num_classes + 5])
        conv_dims = K.cast(K.reshape(conv_dims, [1,1,1,1,2]), K.dtype(feats))

        box_confidence = K.sigmoid(feats[..., 4:5])
        box_xy = K.sigmoid(feats[...,:2])
        box_wh = K.exp(feats[...,2:4])
        box_class_probs = K.softmax(feats[...,5:])
        return box_confidence, box_xy, box_wh, box_class_probs

    def yolo_filter_boxes(self,box_confidence, boxes, box_class_probs, threshold=.6):
        box_scores = box_confidence * box_class_probs
        box_classes = K.argmax(box_scores, axis=-1)
        box_class_scores = K.max(box_scores, axis=-1)

        filtering_mask = box_class_scores >= threshold

        scores = tf.boolean_mask(box_class_scores, filtering_mask)
        boxes = tf.boolean_mask(boxes, filtering_mask)
        classes = tf.boolean_mask(box_classes, filtering_mask)

        return scores, boxes, classes

    def yolo_non_max_suppression(self,scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
        max_boxes_tensor = K.variable(max_boxes, dtype='int32')  # tensor to be used in tf.image.non_max_suppression()
        K.get_session().run(tf.variables_initializer([max_boxes_tensor]))  # initialize variable max_boxes_tensor

        nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold=iou_threshold)

        scores = K.gather(scores, nms_indices)
        boxes = K.gather(boxes, nms_indices)
        classes = K.gather(classes, nms_indices)

        return scores, boxes, classes

    def scale_boxes(delf,boxes, image_shape):
        """ Scales the predicted boxes in order to be drawable on the image"""
        height = image_shape[0]
        width = image_shape[1]
        image_dims = K.stack([height, width, height, width])
        image_dims = K.reshape(image_dims, [1, 4])
        boxes = boxes * image_dims
        return boxes

    def yolo_eval(self,yolo_outputs, image_shape=(720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
        box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
        boxes = yolo_boxes_to_corners(box_xy, box_wh)

        scores, boxes, classes = self.yolo_filter_boxes(box_confidence, boxes, box_class_probs, score_threshold)

        # Scale boxes back to original image shape.
        boxes = self.scale_boxes(boxes, image_shape)

        scores, boxes, classes = self.yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)

        return scores, boxes, classes

    def draw_boxes(delf,image, out_scores, out_boxes, out_classes, class_names, colors):

        font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                                  size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300

        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            label = '{} {:.2f}'.format(predicted_class, score)

            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            print(label, (left, top), (right, bottom))

            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                draw.rectangle([left + i, top + i, right - i, bottom - i], outline=colors[c])
            draw.rectangle([tuple(text_origin), tuple(text_origin + label_size)], fill=colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw

    def generate_colors(self,class_names):
        hsv_tuples = [(x / len(class_names), 1., 1.) for x in range(len(class_names))]
        colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))
        random.seed(10101)  # Fixed seed for consistent colors across runs.
        random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
        random.seed(None)  # Reset seed to default.
        return colors


    def detect(self, image_file, model_image_size = (608, 608)):
        # Preprocess your image
#         resized_image = image.resize(tuple(reversed(model_image_size)), Image.BICUBIC)
#         image_data = np.array(resized_image, dtype='float32')
#         image_data /= 255.
#         image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
        image, image_data = preprocess_image(image_file, models_image_size=(608,608))

        out_scores, out_boxes, out_classes = self.sess.run([self.scores, self.boxes, self.classes],
                                                      feed_dict={self.yolo_model.input: image_data, K.learning_phase(): 0})
        # Generate colors for drawing bounding boxes.
        colors = self.generate_colors(self.class_names)
        # Draw bounding boxes on the image file
        self.draw_boxes(image, out_scores, out_boxes, out_classes, self.class_names, colors)

        return image

    def close_session(self):
        self.sess.close()
        

if __name__ == '__main__':
    yolo = YOLO()
    image = yolo.detect("F:\\car.JPG")
    image.save("F:\\result.JPG", quality=100)
#     cv.namedWindow("camera", 1)

#     capture = cv.VideoCapture(0)            #开启摄像头
#     num = 0;
#     while True:
#         result, img = capture.retrieve()
#         image = Image.fromarray(cv.cvtColor(img,cv.COLOR_BGR2RGB))
#         image = yolo.detect(image)
#         image.save("test.jpg", quality=100)
#         im = cv.cvtColor(np.asarray(image),cv.COLOR_RGB2BGR)
#         cv.imshow("camera", im)
#         key = cv.waitKey(100)

    yolo.close_session()
#     del (capture)
#     cv.DestroyWindow("camera")

ValueError: bad marshal data (unknown type code)